In [74]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers
import json
import pandas as pd

In [75]:
es = elasticsearch.Elasticsearch()

# Histogram

In [76]:
query={
        "aggs": {
                "whole_ratings": {
                        "histogram":{
                                "field":"rating",
                                "interval":1.0
                        }
                }
        }
}

In [77]:
if es is not None:
    res=(es.search(index='ratings', body=json.dumps(query)))
    print(res['aggregations'])   

{'whole_ratings': {'buckets': [{'key': 0.0, 'doc_count': 1101}, {'key': 1.0, 'doc_count': 5013}, {'key': 2.0, 'doc_count': 11720}, {'key': 3.0, 'doc_count': 30602}, {'key': 4.0, 'doc_count': 36473}, {'key': 5.0, 'doc_count': 15095}]}}


# Time Series

In [78]:
query={
        "aggs": {
                "timestamp": {
                        "date_histogram": {
                                "field":"@timestamp",
                                "interval":"hour"
                        }
                }
        }
}

In [82]:
if es is not None:
    res=(es.search(index='logstash-2017.05.04', body=json.dumps(query)))
    print(res['aggregations']) 

{'timestamp': {'buckets': [{'key_as_string': '2017-05-04T00:00:00.000Z', 'key': 1493856000000, 'doc_count': 1432}, {'key_as_string': '2017-05-04T01:00:00.000Z', 'key': 1493859600000, 'doc_count': 1036}, {'key_as_string': '2017-05-04T02:00:00.000Z', 'key': 1493863200000, 'doc_count': 1242}, {'key_as_string': '2017-05-04T03:00:00.000Z', 'key': 1493866800000, 'doc_count': 790}, {'key_as_string': '2017-05-04T04:00:00.000Z', 'key': 1493870400000, 'doc_count': 740}, {'key_as_string': '2017-05-04T05:00:00.000Z', 'key': 1493874000000, 'doc_count': 1148}, {'key_as_string': '2017-05-04T06:00:00.000Z', 'key': 1493877600000, 'doc_count': 938}, {'key_as_string': '2017-05-04T07:00:00.000Z', 'key': 1493881200000, 'doc_count': 1558}, {'key_as_string': '2017-05-04T08:00:00.000Z', 'key': 1493884800000, 'doc_count': 626}, {'key_as_string': '2017-05-04T09:00:00.000Z', 'key': 1493888400000, 'doc_count': 1690}, {'key_as_string': '2017-05-04T10:00:00.000Z', 'key': 1493892000000, 'doc_count': 576}, {'key_as_s

In [83]:
query={
        "query":{
                "match": {
                        "agent":"Googlebot"
                }
        },
        "aggs":{
                "timestamp":{
                        "date_histogram":{
                                "field":"@timestamp",
                                "interval":"hour"
                        }
                }
        }
}

In [84]:
if es is not None:
    res=(es.search(index='logstash-2017.05.04', body=json.dumps(query)))
    print(res['aggregations'])

{'timestamp': {'buckets': [{'key_as_string': '2017-05-04T00:00:00.000Z', 'key': 1493856000000, 'doc_count': 26}, {'key_as_string': '2017-05-04T01:00:00.000Z', 'key': 1493859600000, 'doc_count': 20}, {'key_as_string': '2017-05-04T02:00:00.000Z', 'key': 1493863200000, 'doc_count': 28}, {'key_as_string': '2017-05-04T03:00:00.000Z', 'key': 1493866800000, 'doc_count': 24}, {'key_as_string': '2017-05-04T04:00:00.000Z', 'key': 1493870400000, 'doc_count': 32}, {'key_as_string': '2017-05-04T05:00:00.000Z', 'key': 1493874000000, 'doc_count': 22}, {'key_as_string': '2017-05-04T06:00:00.000Z', 'key': 1493877600000, 'doc_count': 26}, {'key_as_string': '2017-05-04T07:00:00.000Z', 'key': 1493881200000, 'doc_count': 24}, {'key_as_string': '2017-05-04T08:00:00.000Z', 'key': 1493884800000, 'doc_count': 18}, {'key_as_string': '2017-05-04T09:00:00.000Z', 'key': 1493888400000, 'doc_count': 20}, {'key_as_string': '2017-05-04T10:00:00.000Z', 'key': 1493892000000, 'doc_count': 20}, {'key_as_string': '2017-05-

In [85]:
query={
        "query":{
                "match": {
                        "response":"500"
                }
        },
        "aggs":{
                "timestamp":{
                        "date_histogram":{
                                "field":"@timestamp",
                                "interval":"minute"
                        }
                }
        }
}

In [86]:
if es is not None:
    res=(es.search(index='logstash-2017.05.04', body=json.dumps(query)))
    print(res['aggregations'])

{'timestamp': {'buckets': []}}


# Nested Query

In [103]:
query={
"query": {"match_phrase":{"title":"Star Wars"}},
"aggs":{
"title":{
"terms":{"field":"title.raw"},
"aggs": {
"avg_rating": {
"avg" : {"field":"rating"}
}
}
}
}
}

In [106]:
if es is not None:
    res=(es.search(index='ratings', body=json.dumps(query)))
    print(res['aggregations'])

{'title': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'Star Wars: Episode IV - A New Hope (1977)', 'doc_count': 582, 'avg_rating': {'value': 4.221649484536083}}, {'key': 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'doc_count': 468, 'avg_rating': {'value': 4.232905982905983}}, {'key': 'Star Wars: Episode VI - Return of the Jedi (1983)', 'doc_count': 434, 'avg_rating': {'value': 4.059907834101383}}, {'key': 'Star Wars: Episode I - The Phantom Menace (1999)', 'doc_count': 276, 'avg_rating': {'value': 3.199275362318841}}, {'key': 'Star Wars: Episode II - Attack of the Clones (2002)', 'doc_count': 164, 'avg_rating': {'value': 3.1036585365853657}}, {'key': 'Star Wars: Episode III - Revenge of the Sith (2005)', 'doc_count': 124, 'avg_rating': {'value': 3.629032258064516}}, {'key': 'Star Wars: Episode VII - The Force Awakens (2015)', 'doc_count': 58, 'avg_rating': {'value': 3.5172413793103448}}, {'key': 'Star Wars: The Clone Wars (2008)', 'doc_